# Klassifikation

In diesem Notebook werden verschiedene Klassifikationsmodelle trainiert. Ziel ist es, die Städte im Datensatz aufgrund ihrer Feinstaubbelastung in zwei Klassen einzuteilen. Als Zielvariable wird also die Belastung mit mittelgroßen Feinstaubpartikeln (PM2.5, gemessen in µg/m³) angesetzt. 

Als Schwellenwert werden zwei Ansätze getestet:
1. WHO-Richtline von 5 µg/m³: fachlicher Standard
2. Median: datengetriebene Größe

Als Modelle werden verglichen:
1. Logistische Regression
2. Random Forest
3. Gradient Boosting

Verwendet werden verschiedene Module der Python Bibliothek **Scitkit-learn** für maschinelles Lernen

📌 **Datenstand:** `cleaned_air_quality_data_2025-03-27.csv`  
📁 **Importiert aus:** lokaler Datei (--> gitignore)

## 📚 Inhaltsverzeichnis 
(Diese Art von Inhaltsverzeichnis mit Link funktioniert leider in Notebooks nicht, weil die as JSON gespeichert werden und nicht als HTML...)

- [0. Datensatz laden](#0-datensatz-laden)
- [1. Vollständige Schadstoffmessungen und geographische Verteilung](#1-vollständige-schadstoffmessungen-und-geographische-verteilung)
- [2. Clusterberechnung - mehrstufig](#2-clusterberechnung-mehrstufig)
- [3. Clusterbeschreibung mit PCA](#3-clusterbeschreibung-mit-pca)
- [4. Geographische Verteilung der Schadstoffcluster](#4-geographische-verteilung-der-schadstoffcluster)
- [5. Inhaltliche Interpretation](#5-inhaltliche-interpretation)


# 0. Datensatz laden

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

In [ ]:
df = pd.read_csv("data/cleaned_air_quality_data_2025-03-27.csv")
df.head()

# 1. Dateframe vorbereiten

Als Target benutzen wir mittelgroße Feinstaubpartikel (PM2.5), als Features alle anderen Schadstoffe im Datensatz: CO, NO₂, SO₂, O₃ und (im allerersten Modell) PM10.

Weil das Imputieren von Werten für nicht vorhandenen Kategorien (Schadstoffen) für eine Stadt nicht mehr als "Raten" ist, werden nur Städte in die Analyse mit aufgenommen, für die Messwerte zu allen sechs Luftschadstoffen vorliegen. (--> Wiebke: Was ich meine, ist: Ich kann innerhalb einer Stadt Werte imputieren, aber es ist ziemlicher Quatsch, die Werte von einer Stadt auf die andere zu übertragen, also stadtübergreifend zu imputieren.)

Die Luftqualität kann auch auf dem arithmetischen Mittel berechnet werden. Dies hat den Vorteil, dass die natürliche Varianz besser abgebildet wird und den Nachteil, dass Machine-Learning-Modelle sich bei der Klassifikation schlechter abschneiden. Beide Varianten wurden komplett durchgerechnet. Da hier das Verhalten unterschiedlicher Klassifikationsmodelle gezeigt werden soll, wurde für die finale Analyse der Median gewählt.

Um ein Klassifikationsmodell zur Vorhersage der Luftqualität von Städten zu erstellen, wird eine Zielvariable mit dem Namen **AirQualityLabel** eingeführt. Diese ordnet jeder Stadt eine von zwei Klassen zu:

- 0 → Gute Luftqualität

- 1 → Schlechte Luftqualität

Die Luftqualität wird aus den Medianen der einzelnen Schadstoffe pro Stadt berechnet.


In [ ]:
# Liste relevanter Schadstoffe
pollutants = ['Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2']

In [ ]:
# Mittelwerte pro Stadt berechnen (Index = City)
df_median = df.groupby('City')[pollutants].median()

In [ ]:
# Für jede Stadt: Wie viele Mittelwerte sind vorhanden?
df_median['Num_Valid_Pollutants'] = df_median[pollutants].notna().sum(axis=1)

# Übersicht: Wie viele Städte haben wie viele gültige Schadstoffwerte?
coverage_summary = df_median['Num_Valid_Pollutants'].value_counts().sort_index()

# Ergebnis anzeigen
coverage_summary

In [ ]:
# Nur Städte mit allen 6 Schadstoff-Mittelwerten

df_median_complete = df_median[df_median['Num_Valid_Pollutants'] == 6]

len(df_median_complete)

Der Dataframe enthält nun noch 406 Städte, für die jeweils Wertte für jeden Schadstoff vorliegen.

# 2. WHO-Richtlinie

Als erstes wird die Klasseneinteilung auf der Grundlage einen fachlichen Standards, nämlich des aktuellen WHO-Grenzwerts für PM2.5 von 5 µg/m³ vorgenommen:

In [ ]:
# Anzahl Städte mit guter/schlechter Luft (nach WHO-Grenzwert)
(df_median_complete['Pm25'] <= 5).value_counts()

In [ ]:
# Städte mit PM2.5 ≤ 5 µg/m³ filtern
clean_cities = df_median_complete[df_median_complete['Pm25'] <= 5]

# Ergebnis anzeigen
clean_cities

In [ ]:
# Anzahl gültiger PM2.5-Werte pro Stadt
pm25_counts = df.groupby('City')['Pm25'].count().sort_values()

# Zeige nur die "sauberen" Städte
pm25_counts.loc[['Plovdiv', 'Yazd']]

Dem WHO-Grenzwert entsprechen in unserem Datensatz nur zwei von 406 Städten: Plovdiv (Bulgarien), Yazd (Iran). Bei genauerem Hinschauen fällt allerdings auf, dass es für Plovidiv nur einen einzigen Messwert gibt und für Yazd nur sehr wenige. Die Messwerte sind damit nicht aussagekräftig.

Die WHO-Richtline kann also für das Training von Klassifikationsmodellen nicht als Schwellenwert verwendet werden - es hätte keine zweite Klasse, von der es lernen könnte.

**OFFENE FRAGE**: Ist es gut, für die restliche Analyse alle Städte drinzulassen, auch wenn sie nur wenige Messwerte pro Schadstoff haben? Sollte man da noch was aussortieren? Oder einfach mal so lassen, weil in der Realität die Datenqualität auch nur selten optimal ist?

# 3. Median der Zielvariablen

Als Alternative wird der Median als datengetriebener Grenzwert gewählt. Die Einteilung in Städte mit guter und schlechter Luftqualität basiert also auf dem Medianwert der durchschnittlichen PM2.5-Konzentration aller Städte im Datensatz. 

Durch die Verwendung des Medians entsteht eine ausgewogene Verteilung zwischen den beiden Klassen, die ein stabiles Training und eine faire Bewertung des Modells ermöglicht.

Die Berechnung erfolgte folgendermaßen:

    pm25_median = df_median['Pm25'].median()
    df_median['AirQualityLabel'] = (df_median['Pm25'] > pm25_median).astype(int)

 Städte mit einem PM2.5-Mittelwert über dem Median werden als "schlechte Luftqualität" (1) klassifiziert, alle anderen als "gute Luftqualität" (0).



In [ ]:
# Median von PM2.5 berechnen
pm25_median = df_median['Pm25'].median()

# Zielvariable hinzufügen
df_median['AirQualityLabel'] = (df_median['Pm25'] > pm25_median).astype(int)

In [ ]:
# Umgang mit NaN-Werten

# Anzahl fehlender Werte pro Spalte
df[["City"] + pollutants].isna().sum()

In [ ]:
# Anzahl gültiger Werte pro Stadt und Schadstoff
df.groupby('City')[pollutants].count().sort_values(by='Pm25')


Weil das Imputieren von überhaupt nicht vorhandenen Kategorien (Schadstoffen) für eine Stadt eigentlich nicht mehr als "Raten" ist, machen wir das hier nicht und reduzieren die Trainingsdaten auf die 404 Städte, die für alle Schadstoffe Werte gemeldet haben.

In [ ]:
df_median_complete.head()

In [ ]:
df_median_complete.shape

In [ ]:
features = ['Co', 'No2', 'O3', 'Pm10', 'So2']
X = df_median_complete[features]
y = df_median_complete['AirQualityLabel']

In [ ]:
# Wir haben  aktuell 404 Städte in df_median_complete. Davon nehmen wir 80% für das Training und 20% für den Test.
# Wir verwenden den Standard-Trainings-Test-Split von scikit-learn.

# Split in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Modell 1: Logistic Regression

In [ ]:
#  Modelltraining
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Vorhersagen & Bewertung
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Wenn wir PM10 als Feature drinlassen, klassifiziert das Modell prima - und das ist auch zu erwarten, weil PM2.5 eine Teilmenge von PM10 ist. Also lassen wir PM10 jetzt mal raus und schauen, was dann passiert:

In [ ]:
features_no_PM10 = ['Co', 'No2', 'O3', 'So2']
X = df_median_complete[features_no_PM10]
y = df_median_complete['AirQualityLabel']

In [ ]:
# Split in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelltraining
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Vorhersagen & Bewertung
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


In [ ]:
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(cm, 
                     index=["Tatsächlich Negativ", "Tatsächlich Positiv"], 
                     columns=["Vorhergesagt Negativ", "Vorhergesagt Positiv"])

print(cm_df)

In [ ]:
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.ylabel("Tatsächliche Werte")
plt.xlabel("Vorhergesagte Werte");

## 🧾 Entscheidung zur Feature-Auswahl: PM10 aus den Features entfernen

Um die Luftqualität in Städten zu klassifizieren, wurden zunächst alle Schadstoffe (CO, NO₂, O₃, PM10, PM2.5, SO₂) als Features in das Modell aufgenommen, obwohl PM10 und PM2.5 naturgemäß eine hohe Korrelation aufweisen (über 97%).

🔹 Test mit PM10 als Feature:

Das Modell erzielte eine hohe Genauigkeit von 91% und einen sehr hohen F1-Score von 0.92 für die Vorhersage von schlechter Luft (Klasse 1).

Präzision und Recall bei der Klassifikation von „schlechter Luft“ waren sehr hoch (nahe 1.0), was das Modell besonders präzise bei der Vorhersage von schlechter Luft machte.

Allerdings zeigte sich auch, dass das Modell zwischen den Klassen kaum differenzierte, was die klassenspezifische Leistung bei guter Luft (Klasse 0) beeinträchtigte.

🔹 Test ohne PM10 als Feature:

Nachdem PM10 entfernt wurde, ging die Genauigkeit leicht zurück auf 80% und der F1-Score für schlechte Luft sank von 0.92 auf 0.83.

Das Modell zeigt nun jedoch eine ausgewogenere Klassifikation (bevorzogt nicht mehr das Label "schlechte Luft") und ist weniger von redundanten Informationen beeinflusst.

🧠 Ergebnis:

Die Tests haben gezeigt, dass das Modell besser differenziert, wenn PM10 aus den Features entfernt wird. Auch wenn die Gesamtgenauigkeit etwas zurückgeht, ist das Modell nun besser in der Lage, zwischen guter und schlechter Luft zu unterscheiden.
Die Entscheidung, PM10 aus den Features zu entfernen, basiert auf der starken Korrelation mit PM2.5, die zu einer Redundanz führte und das Modell verzerrte.



## Modell 2: Random Forest

In [ ]:
# Features and train test split repeated from previous model, just to make clear what is being used:

features_no_PM10 = ['Co', 'No2', 'O3', 'So2']
X = df_median_complete[features_no_PM10]
y = df_median_complete['AirQualityLabel']

# Split in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Random Forest-Modell erstellen
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Modell trainieren
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)


In [ ]:
# Vorhersagen & Bewertung drucken
print(classification_report(y_test, y_pred_rf))
# print(confusion_matrix(y_test, y_pred_rf))

In [ ]:
cm = confusion_matrix(y_test, y_pred_rf)

cm_df = pd.DataFrame(cm, 
                     index=["Tatsächlich Negativ", "Tatsächlich Positiv"], 
                     columns=["Vorhergesagt Negativ", "Vorhergesagt Positiv"])

print(cm_df)

In [ ]:
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.ylabel("Tatsächliche Werte")
plt.xlabel("Vorhergesagte Werte");

In [ ]:
# Feature importance extrahieren
importances = rf_model.feature_importances_

In [ ]:
# Feature-Importanz in ein DataFrame umwandeln
feature_importance_df = pd.DataFrame({
    'Feature': features_no_PM10,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Feature-Importanz anzeigen
plt.figure(figsize=(8, 4))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df, palette='viridis')
plt.title('Feature Importance (Random Forest)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()



Am wichtigsten für die Klassifizierung ist das Feature "CO". Kann es sein, dass die falsch klassifizierten Städte auffällige CO-Werte haben?

In [ ]:
# Falsch klassifizierte Städte finden
incorrect_predictions = X_test.copy()
incorrect_predictions['True Label'] = y_test
incorrect_predictions['Predicted Label'] = y_pred_rf

# Nur falsch klassifizierte Städte herausfiltern
incorrect_predictions = incorrect_predictions[incorrect_predictions['True Label'] != incorrect_predictions['Predicted Label']]

# CO-Werte der falsch klassifizierten Städte
incorrect_predictions['CO'] = X_test.loc[incorrect_predictions.index, 'Co']

# Ausgabe der Städte mit ihren CO-Werten
incorrect_predictions[['True Label', 'Predicted Label', 'CO']]


In [ ]:
# CO-Werte der falsch klassifizierten Städte
co_values = incorrect_predictions['CO']

# Berechne die wichtigsten Statistiken (Durchschnitt, IQR)
co_mean = co_values.mean()
co_std = co_values.std()
co_min = co_values.min()
co_max = co_values.max()

# Berechne Interquartilsabstand (IQR)
Q1 = co_values.quantile(0.25)
Q3 = co_values.quantile(0.75)
IQR = Q3 - Q1

# Anzeigen der CO-Statistiken
print(f"Durchschnittlicher CO-Wert: {co_mean:.2f}")
print(f"Standardabweichung: {co_std:.2f}")
print(f"Minimaler CO-Wert: {co_min:.2f}")
print(f"Maximaler CO-Wert: {co_max:.2f}")
print(f"Interquartilsabstand (IQR): {IQR:.2f}")


Hier muss man noch nachdenken, wie man das Modell verbessern kann. Man sollte die falsch zugeordneten Städte überprüfen, ob da mit den Daten was nicht stimmt. Und dann noch die Hyperparameter besser einstellen und so. Nicht mehr heute.

## Modell 3: Gradient Boosting

In [ ]:
# Gradient Boosting Modell erstellen
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Modell trainieren
gb_model.fit(X_train, y_train)

# Vorhersagen & Bewertung
y_pred_gb = gb_model.predict(X_test)
print(classification_report(y_test, y_pred_gb))
# print(confusion_matrix(y_test, y_pred_gb))


In [ ]:
cm = confusion_matrix(y_test, y_pred_gb)

cm_df = pd.DataFrame(cm, 
                     index=["Tatsächlich Negativ", "Tatsächlich Positiv"], 
                     columns=["Vorhergesagt Negativ", "Vorhergesagt Positiv"])

print(cm_df)

In [ ]:
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.ylabel("Tatsächliche Werte")
plt.xlabel("Vorhergesagte Werte");